In [2]:
#1. Create a python method that takes arguments int X and int Y,and updates DEPART and RETURN fields in test_payload1.xml:
# DEPART gets set to X days in the future from the current date (whatever the current date is at the moment of executing the code)
# RETURN gets set to Y days in the future from the current date

import datetime
import xml.etree.ElementTree as ET
FILE_PATH = 'data/test_payload1.xml'

def updateDepartureReturnDates(X,Y):
    '''
    X the number of days to be added to Departure date
    Y the number of days to be added to Return date
    '''
    tree = ET.parse(FILE_PATH)
    root = tree.getroot()
    dateFormat = "%m/%d/%Y"
    today = datetime.date.today()
    for departs in root.iter('DEPART'):
        new_depart_date = (today + datetime.timedelta(days=X)).strftime(dateFormat)
        departs.text = str(new_depart_date)
    for returns in root.iter('RETURN'):
        new_return_date = (today + datetime.timedelta(days=Y)).strftime(dateFormat)
        returns.text = str(new_return_date)
    print(f"The Departure date ({new_depart_date}) and Return date ({new_return_date}) have been updated.")
    tree.write(FILE_PATH)
updateDepartureReturnDates(2,3)

The Departure date (02/28/2021) and Return date (03/01/2021) have been updated.


In [655]:
#2. Create a python method that takes a json element
#as an argument, and removes that element from test_payload.json.

import json

DEST_FILE = 'data/test_payload_updated.json'
SOURCE_FILE = 'data/test_payload.json'

def removeElement(delElement):
    '''
    Gets an argument (delElement) as the element to be deleted 
    '''
    with open(DEST_FILE, 'w') as dest_file:
        with open(SOURCE_FILE, 'r') as source_file:
            for line in source_file:
                element = json.loads(line.strip())
                if delElement in element:
                    del element[delElement]
                dest_file.write(json.dumps(element))
                
removeElement('inParams')

In [656]:
#3. Create a python script that parses jmeter log files,
#and in the case if there are any non-successful endpoint responses recorded in the log,
#prints out the label, response code, response message, failure message,
#and the time of non-200 response in human-readable format in PST timezone
#(e.g. 2021-02-09 06:02:55 PST).
#Please use Jmeter_log1.jtl, Jmeter_log2.jtl as input files for testing out your script.

import pandas as pd
import datetime
import pytz

FILE_PATH = "data/Jmeter_log1.jtl"
#FILE_PATH = "data/Jmeter_log2.jtl"

def parseLogFile(FILE_PATH):
    def convertTS2DateTime(value):
            dt = datetime.datetime.fromtimestamp(int(value)/1000, pytz.timezone("US/Pacific"))
            return dt.strftime("%Y-%m-%d %H:%M:%S %Z")

    df = pd.read_csv(FILE_PATH)
    df = df[df['responseCode'] != 200]
    df['timeStamp'] = df['timeStamp'].apply(lambda x: convertTS2DateTime(x))
    return df[['label','responseCode','responseMessage','failureMessage','timeStamp']]

parseLogFile(FILE_PATH)

,label,responseCode,responseMessage,failureMessage,timeStamp
4,Direct_test_Services_QuoteRequest,504,Gateway Time-out,NaN,2021-02-09 06:01:23 PST
5,Internal_Quote_Travel,504,Gateway Timeout,NaN,2021-02-09 06:01:23 PST
6,Proxy_test_Services_QuoteRequest,504,Gateway Timeout,NaN,2021-02-09 06:01:30 PST
24,Proxy_test_Services_QuoteRequest,504,Gateway Timeout,NaN,2021-02-09 06:02:55 PST
25,Internal_Quote_Travel,504,Gateway Timeout,NaN,2021-02-09 06:03:04 PST


In [657]:
#version 2

import datetime
import pytz

FILE_PATH = "data/Jmeter_log1.jtl"
#FILE_PATH = "data/Jmeter_log2.jtl"

def parseLogFile(FILE_PATH):
    
    with open(FILE_PATH) as f:
        next(f) #ignore the first line
        for line in f:
            values = line.split(",")
            if values[3] is not None and values[3] == '200':
                continue

            #millisecond-precise timestamp. So, divide it by 1000
            dt = datetime.datetime.fromtimestamp(int(values[0])/1000, pytz.timezone("US/Pacific"))
            if values[2] is not None : print(values[2], end = ", ")
            if values[3] is not None :print(values[3], end = ", ")
            if values[4] is not None :print(values[4], end = ", ")
            if values[8] is not None :print(values[8], end = ",")
            print(dt.strftime("%Y-%m-%d %H:%M:%S %Z"),end = "\n")
parseLogFile(FILE_PATH)

Direct_test_Services_QuoteRequest, 504, Gateway Time-out, ,2021-02-09 06:01:23 PST
Internal_Quote_Travel, 504, Gateway Timeout, ,2021-02-09 06:01:23 PST
Proxy_test_Services_QuoteRequest, 504, Gateway Timeout, ,2021-02-09 06:01:30 PST
Proxy_test_Services_QuoteRequest, 504, Gateway Timeout, ,2021-02-09 06:02:55 PST
Internal_Quote_Travel, 504, Gateway Timeout, ,2021-02-09 06:03:04 PST
